In [1]:
%matplotlib notebook
#%matplotlib widget

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator
from scipy.interpolate import interp1d, RBFInterpolator
from scipy.spatial import distance_matrix

def simple_idw_squared(x, z, xi):
    dist = distance_matrix(x[:, np.newaxis], xi[:, np.newaxis]) 
    weights = 1.0 / (dist ** 2)  # Square the distances
    weights /= weights.sum(axis=0)
    zi = np.dot(weights.T, z)
    return zi

# 1. Create the base cosine wave
x = np.linspace(-5 - np.pi/2, 4 - np.pi/2, 10000)
base_z = np.cos(x) 

# 2. Create linspace for perpendicular points
x_perp = np.linspace(-5 - np.pi/2, 4 - np.pi/2, 10000)

# Setup plot
original_figsize = (10, 6) 
new_figsize = (original_figsize[0] * 1.2, original_figsize[1] * 1.2) 
fig, ax = plt.subplots(figsize=new_figsize)
line_sine, = ax.plot(x, base_z, 'b-', label='Floor Surface -> cos(x)')
interp_line, = ax.plot([], [], 'g-', label='Roof Surface -> 0.5 True Thickness ')
perp_lines = []

# Custom x-axis ticks and labels (integers only)
ax.set_xticks(np.arange(-5, 5))  # Generate integer ticks from -5 to 5
ax.set_xticklabels(np.arange(-5, 5), fontsize=10)  # Optional: Adjust fontsize

# Custom y-axis ticks and labels (integers only)
ax.set_yticks(np.arange(-6, 3))   # Generate integer ticks from -6 to 3
ax.set_yticklabels(np.arange(-6, 3), fontsize=10)  # Optional: Adjust fontsize

# Add minor y-axis ticks at half-integer positions
ax.yaxis.set_minor_locator(MultipleLocator(0.5))  

ax.set_xlim(-4 - np.pi/2, 4 - np.pi/2) 
ax.set_ylim(-7.0, 2.5)  # Adjusted z-limits to accommodate shifted lines
ax.set_aspect('equal')
ax.set_xlabel("X")
ax.set_ylabel("Z")
ax.set_title("Regional Compression of Surfaces")
ax.legend()
ax.grid(True)

# Initialize perpendicular lines (representing true thickness)
for i in range(len(x_perp)):
    line, = ax.plot([], [], 'r-', linewidth=0.3)
    perp_lines.append(line)

# Define extra lines parameters (initial position and final angle deviation)
np.random.seed(42)
x_start_points = np.random.rand(10) * 8 - 2 - np.pi
x_start_points.sort()

extra_lines_data = [
    {'x_start': x_start_points[0], 'final_deviation': 15},
#    {'x_start': x_start_points[1], 'final_deviation': 15},
    {'x_start': x_start_points[2], 'final_deviation': -15},
    {'x_start': x_start_points[3], 'final_deviation': 25},
#    {'x_start': x_start_points[4], 'final_deviation': -10},
    {'x_start': x_start_points[5], 'final_deviation': 0},
    {'x_start': x_start_points[6], 'final_deviation': 15},
    {'x_start': x_start_points[7], 'final_deviation': 20},
    {'x_start': x_start_points[8], 'final_deviation': 70},
#    {'x_start': x_start_points[9], 'final_deviation': 15},
]

# Initialize extra line objects
extra_lines = []
extra_lines_duplicates = [] 
pink_lines = [] 
purple_lines = [] 
green_lines = []
green_lines_tt = []
for _ in extra_lines_data:
    line, = ax.plot([], [], 'k-', linewidth=2)
    extra_lines.append(line)
    duplicate_line, = ax.plot([], [], 'k-', linewidth=2) 
    extra_lines_duplicates.append(duplicate_line)
    # Initialize pink and purple lines (initially invisible)
    pink_line, = ax.plot([], [], 'm-', linewidth=2) 
    pink_lines.append(pink_line)
    purple_line, = ax.plot([], [], 'purple', linewidth=2) 
    purple_lines.append(purple_line)
    green_line, = ax.plot([], [], 'black', linewidth=3) 
    green_lines.append(green_line)
    green_line_tt, = ax.plot([], [], 'green', linestyle='dashed', linewidth=0.1) 
    green_lines_tt.append(green_line_tt)

# Initialize text annotations for line lengths
extra_line_texts = []
pink_line_texts = []
purple_line_texts = []
green_line_texts = []
for line_data in extra_lines_data:
    x_start = line_data['x_start']
    extra_line_text = ax.text(x_start, -2.1, '', ha='left', va='top', rotation=-35, fontsize=7)
    extra_line_texts.append(extra_line_text)
    pink_line_text = ax.text(x_start, -4.1, '', ha='left', va='top', rotation=-35, fontsize=7)
    pink_line_texts.append(pink_line_text)
    purple_line_text = ax.text(x_start, -5.1, '', ha='left', va='top', rotation=-35, fontsize=7)
    purple_line_texts.append(purple_line_text)
    green_line_text = ax.text(x_start, -6.6, '', ha='left', va='top', rotation=-35, fontsize=7)
    green_line_texts.append(green_line_text)

# Calculate amplitude based on frame number, with steps and pause
fps = 60
amplitude_step = 0.05
total_steps = int((1.0 - 0.05) / amplitude_step) 
frames_per_step = 1 
total_animation_frames = total_steps * frames_per_step
pause_frames_1 = int(0.1 * fps)  
pause_frames_2 = int(0.1 * fps)  
transition_frames = 1 * 60
surface_separation = 0.5

# Add annotations on the right y-axis
ax.text(1.02, -2, 'z-delta', transform=ax.get_yaxis_transform(), ha='left', va='center')
ax.text(1.02, -4, 'Borehole length', transform=ax.get_yaxis_transform(), ha='left', va='center')
ax.text(1.02, -5, 'True Thickness', transform=ax.get_yaxis_transform(), ha='left', va='center')
ax.text(1.02, -6.5, 'Apparent Vertical Thickness', transform=ax.get_yaxis_transform(), ha='left', va='center')

# Initialize IDW and RBF interpolated lines 
idw_line, = ax.plot([], [], 'b--', label='IDW Curve', linewidth=1)
rbf_line_pink, = ax.plot([], [], 'm--', label='RBF Curve (Pink)', linewidth=1) 
rbf_line_purple, = ax.plot([], [], 'purple', linestyle='--', label='RBF Curve (Purple)', linewidth=1)

# Animation function with amplitude steps, pause, and extra lines
def animate(frame):
    if frame < total_animation_frames:
        amplitude = 0.05 + (frame // frames_per_step) * amplitude_step 
    else:
        amplitude = 1.0

    # Calculate z values for the current amplitude
    z = amplitude * base_z

    # 3. Calculate perpendicular points and update lines
    x_perp_final = []
    z_perp_final = []
    perp_line_lengths = []  # Store the lengths of the perpendicular lines
    for i in range(len(x_perp)):
        idx = np.argmin(np.abs(x - x_perp[i]))
        m_tangent = -amplitude * np.sin(x[idx])

        if m_tangent >= 0:
            perp_angle = np.arctan(-1 / m_tangent)
        else:
            perp_angle = np.arctan(-1 / m_tangent) + np.pi

        if m_tangent == 0:
            perp_angle = np.pi / 2

        x1, z1 = x_perp[i], z[idx]
        x2 = x1 - surface_separation * np.cos(perp_angle)
        z2 = z1 - surface_separation * np.sin(perp_angle)

        x_perp_final.append(x2)
        z_perp_final.append(z2)

        # Calculate and store the length of the perpendicular line
        perp_line_length = np.sqrt((x2 - x1)**2 + (z2 - z1)**2)
        perp_line_lengths.append(perp_line_length)

        if i % 250 == 0:
            perp_lines[i].set_data([x1, x2], [z1, z2])
        else:
            perp_lines[i].set_data([], [])

    # 4. Interpolate the perpendicular points 
    f = interp1d(x_perp_final, z_perp_final, kind='cubic')
    x_interp = np.linspace(min(x_perp_final), max(x_perp_final), 2000)
    z_interp = f(x_interp)

    # Update plot
    line_sine.set_ydata(z)
    interp_line.set_data(x_interp, z_interp)

    # Add extra lines and animate their duplicates downwards after the pause
    if frame >= total_animation_frames:
        transition_progress = min(1.0, (frame - total_animation_frames) / transition_frames)
        # Initialize lists to store points for IDW interpolation
        idw_points_x = []
        idw_points_z = []
        rbf_points_pink_x = []
        rbf_points_pink_z = []
        rbf_points_purple_x = []
        rbf_points_purple_z = []
        
        for i, line_data in enumerate(extra_lines_data):
            x_start = line_data['x_start']
            final_deviation = np.radians(line_data['final_deviation'])

            z_start = amplitude * base_z[np.argmin(np.abs(x - x_start))]

            # Calculate the angle of the vertical line at x_start
            vertical_angle = np.pi/2 if x_start <= 0 else -np.pi/2

            # Calculate the final angle of the extra line relative to the vertical at x_start
            final_angle = vertical_angle - final_deviation

            # Find the intersection point with the interpolated curve
            distances = np.abs(z_interp - (np.tan(final_angle) * (x_interp - x_start) + z_start)) / np.sqrt(1 + np.tan(final_angle)**2)
            idx_closest = np.argmin(distances)
            x_end = x_interp[idx_closest]
            z_end = z_interp[idx_closest]

            # Update the original extra line
            extra_lines[i].set_data([x_start, x_end], [z_start, z_end])
            extra_line_length = z_end - z_start
            extra_line_texts[i].set_text(f"{extra_line_length:.3f}")
            
            # Shift the duplicate line downwards smoothly
            shift_amount = z_start + 2.0
            z_start_duplicate = z_start - shift_amount * transition_progress
            z_end_duplicate = z_end - shift_amount * transition_progress

            # Calculate the x_end for the duplicate line to maintain the final_angle
            x_end_duplicate = x_start + (z_end_duplicate - z_start_duplicate) / np.tan(final_angle)

            # Update the duplicate extra line
            extra_lines_duplicates[i].set_data([x_start, x_end_duplicate], [z_start_duplicate, z_end_duplicate])

            # Update the pink lines and their text annotations 
            if transition_progress == 1.0 and frame == total_frames - 1:
                # Collect points for IDW interpolation after the transition is complete
                idw_points_x.append(x_end_duplicate)
                idw_points_z.append(z_end_duplicate) 
                
                #  borehole length lines are pink
                for i, line_data in enumerate(extra_lines_data):
                    x_start = line_data['x_start']

                    # Calculate z_start based on the current amplitude
                    z_start = amplitude * base_z[np.argmin(np.abs(x - x_start))]

                    # Find the intersection point with the interpolated curve at final_angle (when amplitude is 1.0)
                    final_deviation = np.radians(line_data['final_deviation'])
                    vertical_angle = np.pi / 2 if x_start <= 0 else -np.pi / 2
                    final_angle = vertical_angle - final_deviation

                    distances = np.abs(z_interp - (np.tan(final_angle) * (x_interp - x_start) + z_start)) / np.sqrt(1 + np.tan(final_angle) ** 2)
                    idx_closest = np.argmin(distances)
                    x_end = x_interp[idx_closest]
                    z_end = z_interp[idx_closest]

                    # Calculate the length of the extra line at full amplitude
                    pink_line_length = np.sqrt((x_end - x_start) ** 2 + (z_end - z_start) ** 2)

                    # Update the pink line
                    pink_lines[i].set_data([x_start, x_start], [-4, -4 + pink_line_length])

                    # Update the pink line text annotation
                    pink_line_texts[i].set_text(f"{pink_line_length:.3f}")

                    rbf_points_pink_x.append(x_start)
                    rbf_points_pink_z.append(-4 + pink_line_length) 
                
                # true thickness lines are purple
                for i, line_data in enumerate(extra_lines_data):
                    x_start = line_data['x_start']
                    # Calculate the slope of the tangent at x_start for the current amplitude
                    m_tangent = -amplitude * np.sin(x_start)
                    # Calculate the perpendicular angle
                    if m_tangent >= 0:
                        perp_angle = np.arctan(-1 / m_tangent)
                    else:
                        perp_angle = np.arctan(-1 / m_tangent) + np.pi
                    if m_tangent == 0:
                        perp_angle = np.pi / 2
                    # Calculate z_start on the base cosine wave at full amplitude
                    z_start = 1.0 * base_z[np.argmin(np.abs(x - x_start))]
                    # Find the intersection of this perpendicular line with the interpolated curve
                    distances = np.abs(z_interp - (np.tan(perp_angle) * (x_interp - x_start) + z_start)) / np.sqrt(1 + np.tan(perp_angle)**2)
                    idx_closest = np.argmin(distances)
                    x_end = x_interp[idx_closest]
                    z_end = z_interp[idx_closest]
                    # True thickness is the Euclidean distance between the start and end points of the perpendicular line
                    true_thickness = np.sqrt((x_end - x_start)**2 + (z_end - z_start)**2)
                    # Update the purple line
                    purple_lines[i].set_data([x_start, x_start], [-5, -5 + true_thickness])
                    # Update the purple line text annotation
                    purple_line_texts[i].set_text(f"{true_thickness:.3f}")

                    rbf_points_purple_x.append(x_start)
                    rbf_points_purple_z.append(-5 + true_thickness) 
                
                # Apparent vertical lines are black
                for i, line_data in enumerate(extra_lines_data):
                    x_start = line_data['x_start']

                    # Calculate z_start on the base cosine wave at full amplitude
                    z_start = 1.0 * base_z[np.argmin(np.abs(x - x_start))]
                
                    # Find the index of the point on the interpolated curve with the same x as x_start
                    idx_interp = np.argmin(np.abs(x_interp - x_start))
                
                    # Apparent vertical thickness is the vertical distance to the point with the same x but greater z
                    z_end = z_interp[idx_interp]
                    if z_end > z_start: 
                        apparent_vertical_thickness = z_end - z_start
                    else:
                        apparent_vertical_thickness = 0 
                    
                    # Update the green line
                    green_lines[i].set_data([x_start, x_start], [-6.5, -6.5 + apparent_vertical_thickness])
                    # Update the purple line text annotation
                    green_line_texts[i].set_text(f"{apparent_vertical_thickness:.3f}")

                # true thicnkess concentrated lines are thin green
                x_grid_tt = np.linspace(min(x), max(x), 1000)
                for i in range(0, len(x_grid_tt), 5):   # Iterate with a step of 5
                    x_start = x_grid_tt[i]  # Get the current x_start from x_grid_tt

                    # Calculate z_start on the base cosine wave at full amplitude
                    z_start = 1.0 * base_z[np.argmin(np.abs(x - x_start))]

                    # Find the index of the point on the interpolated curve with the same x as x_start
                    idx_interp = np.argmin(np.abs(x_interp - x_start))

                    # Apparent vertical thickness is the vertical distance to the point with the same x but greater z
                    z_end = z_interp[idx_interp]
                    if z_end > z_start:
                        apparent_vertical_thickness = z_end - z_start
                    else:
                        apparent_vertical_thickness = 0

                    # Update or append to the green lines
                    if i < len(green_lines_tt):  # Update existing lines if available
                        green_lines_tt[i].set_data([x_start, x_start], [-6.5, -6.5 + apparent_vertical_thickness])
                    else:  # Append new lines if needed
                        new_green_line, = ax.plot([x_start, x_start], [-6.5, -6.5 + apparent_vertical_thickness], 'green', linewidth=1)
                        green_lines_tt.append(new_green_line) 

        # 8. IDW-squared interpolation (at y = -2)
        if transition_progress == 1.0 and idw_points_x and idw_points_z:
            # Perform IDW interpolation using your provided simple_idw_squared function
            x_grid = np.linspace(min(x), max(x), 1000)  # Use the same x range as the base wave
            z_idw = simple_idw_squared(np.array(idw_points_x), np.array(idw_points_z), x_grid)
            # Update the IDW curve
            idw_line.set_data(x_grid, z_idw)
        
        if transition_progress == 1.0 and pink_lines:
            x_grid_pink = np.linspace(min(x), max(x), 1000)
            z_rbf_pink = simple_idw_squared(np.array(rbf_points_pink_x), np.array(rbf_points_pink_z), x_grid_pink)
            rbf_line_pink.set_data(x_grid_pink, z_rbf_pink)

        if transition_progress == 1.0 and purple_lines:
            x_grid_purple = np.linspace(min(x), max(x), 1000)
            z_rbf_purple = simple_idw_squared(np.array(rbf_points_purple_x), np.array(rbf_points_purple_z), x_grid_purple)
            rbf_line_purple.set_data(x_grid_purple, z_rbf_purple)

    # Hide extra lines and their duplicates before the pause
    else:
        for line in extra_lines + extra_lines_duplicates:
            line.set_data([], [])
        for line in pink_lines:
            line.set_data([], [])
        for line in purple_lines:
            line.set_data([], [])
        for line in green_lines:
            line.set_data([], [])
        for line in green_lines_tt:
            line.set_data([], [])
        for text in extra_line_texts + pink_line_texts + purple_line_texts + green_line_texts:
            text.set_text('')

    return line_sine, interp_line, *perp_lines, *extra_lines, *extra_lines_duplicates, *pink_lines, *purple_lines, *green_lines_tt, \
           *green_lines, *extra_line_texts, *pink_line_texts, *purple_line_texts, *green_line_texts, idw_line, rbf_line_pink, rbf_line_purple

# Calculate total frames for animation
total_frames = total_animation_frames + pause_frames_1 + pause_frames_2 + transition_frames

# Create and run the animation
ani = animation.FuncAnimation(fig, animate, frames=total_frames, blit=True, interval=1000/600, repeat=False)

# Save the animation as an MP4 video
# ani.save('animation.mp4', writer=animation.FFMpegWriter(fps=60, extra_args=['-vcodec', 'libx264']))

plt.show() 


<IPython.core.display.Javascript object>